In [4]:
from kindra import kindle, constants
import polars as pl 
from polars import col 
import os
from dotenv import load_dotenv

load_dotenv()

False

In [2]:
os.getenv("KINDLE_VOCAB_PATH")

In [3]:
info = kindle.query_db("select * from BOOK_INFO")

OperationalError: unable to open database file